In [1]:
import pandas as pd

In [2]:
# Read Raw Data
raw_incidents = pd.read_excel('raw/Data_Tables_LGA_Criminal_Incidents_Year_Ending_December_2024.xlsx', sheet_name='Table 03', header=0, engine='openpyxl')
raw_offense = pd.read_excel('raw/Data_Tables_LGA_Recorded_Offences_Year_Ending_December_2024.xlsx', sheet_name='Table 03', header=0, engine='openpyxl')

# Process Data

In [3]:
def filter_df(df):
    if df is not None:
        # Keep only Melbourne and Related Crime Data
        filtered_df = df[(df['Local Government Area']=='Melbourne') & 
                         (df['Offence Division']=='A Crimes against the person')
                         ].reset_index(drop=True).copy()
        # Rename Columns
        filtered_df.columns = df.columns.str.replace(r'[^\w]', '_', regex=True).str.lower()
        
        # Add Column offence_id
        filtered_df['offence_id'] = filtered_df['offence_subgroup'].str.split().str[0]

        # Drop Column
        filtered_df = filtered_df.drop(columns=['year_ending'])

        return filtered_df

In [4]:
incidents_df = filter_df(raw_incidents)
offense_df = filter_df(raw_offense)

crime_yearly_df = pd.merge(
    incidents_df,
    offense_df,
    how='outer',
    on=['year', 'local_government_area', 'postcode', 'suburb_town_name', 'offence_id', 'offence_division', 'offence_subdivision', 'offence_subgroup']
)
crime_yearly_df = crime_yearly_df.fillna(0)

# Data Validation

In [5]:
unq_key = ['year','postcode', 'suburb_town_name', 'offence_id']

In [6]:
print("incidents_df")
print(f"row cnt: {incidents_df.shape[0]}")
print(f"unq key cnt: {incidents_df[unq_key].drop_duplicates().shape[0]}")
incidents_df.head()


incidents_df
row cnt: 1972
unq key cnt: 1972


,year,local_government_area,postcode,suburb_town_name,offence_division,offence_subdivision,offence_subgroup,incidents_recorded,offence_id
0,2024,Melbourne,3000,Carlton,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1,A232
1,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,85,A211
2,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,445,A212
3,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",100,A22
4,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,125,A231


In [7]:
print("offense_df")
print(f"row cnt: {offense_df.shape[0]}")
print(f"unq key cnt: {offense_df[unq_key].drop_duplicates().shape[0]}")
offense_df.head()

offense_df
row cnt: 2074
unq key cnt: 2074


,year,local_government_area,postcode,suburb_town_name,offence_division,offence_subdivision,offence_subgroup,offence_count,offence_id
0,2024,Melbourne,3000,Carlton,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1,A232
1,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,83,A211
2,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,443,A212
3,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",146,A22
4,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,141,A231


In [8]:
print("crime_yearly_df")
print(f"row cnt: {crime_yearly_df.shape[0]}")
print(f"unq key cnt: {crime_yearly_df[unq_key].drop_duplicates().shape[0]}")
crime_yearly_df.head()

crime_yearly_df
row cnt: 2080
unq key cnt: 2080


,year,local_government_area,postcode,suburb_town_name,offence_division,offence_subdivision,offence_subgroup,incidents_recorded,offence_id,offence_count
0,2024,Melbourne,3000,Carlton,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1.0,A232,1.0
1,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,85.0,A211,83.0
2,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,445.0,A212,443.0
3,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",100.0,A22,146.0
4,2024,Melbourne,3000,Melbourne,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,125.0,A231,141.0


# Export Data

In [9]:
crime_yearly_df.to_csv('melb_crime_stat_yearly.csv')